In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.compose import ColumnTransformer

In [4]:
df=pd.read_csv('train.csv',usecols=['Age','Fare','Survived'])

In [5]:
df.dropna(inplace=True)

In [6]:
df.head(2)

,Survived,Age,Fare
0,0,22.0,7.2500
1,1,38.0,71.2833


In [12]:
X=df.drop(columns='Survived')
y=df.iloc[:,0]


In [13]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [18]:
clf=DecisionTreeClassifier()
clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)
accuracy_score(y_pred,y_test)

0.6363636363636364

In [19]:
np.mean(cross_val_score(DecisionTreeClassifier(),X,y,cv=10,scoring='accuracy'))

0.6345070422535211

In [24]:
kbin_age = KBinsDiscretizer(n_bins=15,encode='ordinal',strategy='quantile')
kbin_fare = KBinsDiscretizer(n_bins=15,encode='ordinal',strategy='quantile')

In [40]:
trf = ColumnTransformer([
    ('first',kbin_age,[0]),
    ('second',kbin_fare,[1])
])


In [41]:
X_train_trf = trf.fit_transform(X_train)
X_test_trf = trf.transform(X_test)

In [42]:

output = pd.DataFrame({
    'age':X_train['Age'],
    'age_trf':X_train_trf[:,0],
    'fare':X_train['Fare'],
    'fare_trf':X_train_trf[:,1]
})





In [43]:
output['age_labels'] = pd.cut(x=X_train['Age'],
                                    bins=trf.named_transformers_['first'].bin_edges_[0].tolist())
output['fare_labels'] = pd.cut(x=X_train['Fare'],
                                    bins=trf.named_transformers_['second'].bin_edges_[0].tolist())


In [44]:
output.sample(5)

,age,age_trf,fare,fare_trf,age_labels,fare_labels
622,20.0,3.0,15.7417,7.0,"(19.0, 21.0]","(14.454, 18.75]"
291,19.0,3.0,91.0792,13.0,"(16.0, 19.0]","(76.292, 108.9]"
23,28.0,7.0,35.5000,11.0,"(25.0, 28.0]","(31.275, 51.479]"
530,2.0,0.0,26.0000,9.0,"(0.42, 6.0]","(18.75, 26.0]"
394,24.0,5.0,16.7000,7.0,"(23.0, 25.0]","(14.454, 18.75]"


In [45]:
clf = DecisionTreeClassifier()
clf.fit(X_train_trf,y_train)
y_pred2 = clf.predict(X_test_trf)

In [46]:
accuracy_score(y_test,y_pred2)

0.6363636363636364

In [47]:
X_trf = trf.fit_transform(X)
np.mean(cross_val_score(DecisionTreeClassifier(),X,y,cv=10,scoring='accuracy'))

0.6219483568075117

In [56]:
#Without binarization

In [57]:
clf=DecisionTreeClassifier()

In [59]:
clf.fit(X_train,y_train)

y_pred = clf.predict(X_test)

accuracy_score(y_test,y_pred)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').